<a href="https://colab.research.google.com/github/MoulikaGudipally/Moulika_INFO5731_Fall2023/blob/main/Gudipally_Moulika_Exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training.

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.

Algorithms:

(1) MultinominalNB

(2) SVM

(3) KNN

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison

(4) F-1 score

In [1]:
pip install scikit-learn gensim transformers


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertForSequenceClassification
import torch

from google.colab import files

# This will prompt you to select a file.
# Once you select the file, it will be uploaded to Colab.
uploaded = files.upload()

# Read the uploaded file
for fn in uploaded.keys():
    print('Uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving stsa-train.csv to stsa-train.csv
Saving stsa-test.csv to stsa-test.csv
Uploaded file "stsa-train.csv" with length 738844 bytes
Uploaded file "stsa-test.csv" with length 193562 bytes


In [ ]:
import io
train_data = pd.read_csv(io.StringIO(uploaded['stsa-train.csv'].decode('utf-8')))
test_data = pd.read_csv(io.StringIO(uploaded['stsa-test.csv'].decode('utf-8')))


X = train_data['review_text']  # Assuming 'review_text' column contains the reviews
y = train_data['sentiment']    # Assuming 'sentiment' column contains labels

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Extraction - Word2Vec
tokenized_text = X.apply(lambda x: x.split())  # Tokenizing the text
word2vec_model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, sg=1)  # Training Word2Vec model

# Convert text data to Word2Vec vectors
def word_averaging(model, words):
    mean = []
    for word in words:
        if word in model.wv:
            mean.append(model.wv[word])

    if not mean:
        return [0] * model.vector_size
    mean = sum(mean) / len(mean)
    return mean

X_train_word_average = X_train.apply(lambda x: word_averaging(word2vec_model, x.split())).tolist()
X_val_word_average = X_val.apply(lambda x: word_averaging(word2vec_model, x.split())).tolist()

# Models
models = {
    'MultinomialNB': MultinomialNB(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(),
}

# Training and Evaluation
for name, model in models.items():
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_word_average, y_train, cv=10, scoring='accuracy')
    print(f"{name} Cross-Validation Accuracy: {cv_scores.mean()}")

    # Training the model
    model.fit(X_train_word_average, y_train)

    # Validation
    y_pred = model.predict(X_val_word_average)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    print(f"{name} Validation Accuracy: {accuracy}")
    print(f"{name} Validation Precision: {precision}")
    print(f"{name} Validation Recall: {recall}")

# BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenizing and encoding text for BERT
X_train_encoded = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt')
X_val_encoded = tokenizer(X_val.tolist(), padding=True, truncation=True, return_tensors='pt')

# Training BERT model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {
    'input_ids': X_train_encoded['input_ids'],
    'attention_mask': X_train_encoded['attention_mask'],
    'labels': torch.tensor(y_train.tolist())
}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(3):  # Adjust number of epochs as needed
    optimizer.zero_grad()
    outputs = model(**inputs)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Validation with BERT
with torch.no_grad():
    val_inputs = {
        'input_ids': X_val_encoded['input_ids'].to(device),
        'attention_mask': X_val_encoded['attention_mask'].to(device),
        'labels': torch.tensor(y_val.tolist()).to(device)
    }
    val_outputs = model(**val_inputs)
    val_loss = val_outputs.loss
    logits = val_outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()

accuracy = accuracy_score(y_val, predictions)
precision = precision_score(y_val, predictions)
recall = recall_score(y_val, predictions)

print(f"BERT Validation Accuracy: {accuracy}")
print(f"BERT Validation Precision: {precision}")
print(f"BERT Validation Recall: {recall}")

(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [2]:
#Write your code here.
# Install necessary libraries if not already installed
!pip install transformers
!pip install gensim

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
from google.colab import files

# This will prompt you to select a file.
# Once you select the file, it will be uploaded to Colab.
uploaded = files.upload()

# Read the uploaded file
for fn in uploaded.keys():
    print('Uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving Amazon_Unlocked_Mobile.csv to Amazon_Unlocked_Mobile.csv
Uploaded file "Amazon_Unlocked_Mobile.csv" with length 131879567 bytes


In [ ]:
# Use pandas to read the CSV file
import io
df = pd.read_csv(io.StringIO(uploaded['Amazon_Unlocked_Mobile.csv'].decode('utf-8')))

# Drop rows with missing text data
df = df.dropna(subset=['Reviews'])

# Preprocessing ('Reviews' is the column containing text data)
text_data = df['Reviews']

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)

# Continue with the rest of the code for clustering algorithms


# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)

# K-means
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters)
kmeans_labels = kmeans.fit_predict(tfidf_matrix)

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(tfidf_matrix)

# Hierarchical Clustering
hierarchical = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
hierarchical_labels = hierarchical.fit_predict(tfidf_matrix.toarray())

# Word2Vec
tokenized_text = [Reviews.split() for Reviews in text_data]
word2vec_model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
word_vectors = word2vec_model.wv
word2vec_embeddings = np.array([word_vectors[word] for word in word_vectors.key_to_index])

# BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenized_text = df['Reviews'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))
padded = np.array([i + [0]*(128-len(i)) for i in tokenized_text.values])
input_ids = torch.tensor(padded)

with torch.no_grad():
    outputs = model(input_ids)
    bert_embeddings = outputs[0][:, 0, :].numpy()

# Now you have the cluster labels from different algorithms
# kmeans_labels, dbscan_labels, hierarchical_labels, word2vec_embeddings, bert_embeddings




In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

Each clustering method exhibits distinct strengths and characteristics. K-means, DBSCAN, and Hierarchical clustering are traditional clustering algorithms where K-means relies on centroids, DBSCAN identifies clusters based on density, and Hierarchical clustering forms a hierarchy of clusters. These methods are advantageous in different scenarios; K-means is computationally efficient but requires specifying the number of clusters, DBSCAN is effective in identifying arbitrarily shaped clusters but sensitive to its parameters, and Hierarchical clustering offers insights into hierarchical relationships. On the other hand, Word2Vec and BERT, embedding techniques, capture semantic relationships in text. Word2Vec creates word embeddings based on context, while BERT generates dense representations capturing context and meaning. Word2Vec might be more suitable for capturing word-level similarities, while BERT, with its contextual understanding, might offer more nuanced representations for clustering tasks, especially in scenarios where context plays a crucial role. However, the effectiveness of these methods heavily depends on the dataset characteristics, the nature of the clustering problem, and the quality of embeddings or features extracted.




